In [ ]:
import numpy as np

## 付録L　温水床暖房

### L.1 記号及び単位

省略

### L.2 熱損失

$$\large
Q_{loss,rad} = Q_{T,H,rad} \times \frac{(1 - r_{up})}{r_{up}}
$$

$Q_{T,H,rad}$：1 時間当たりの放熱器の処理暖房負荷(MJ/h)  
$r_{up}$：温水床暖房の上面放熱率  

In [ ]:
# 放熱器の熱損失 (1)
def get_Q_loss_rad(Q_T_H_rad, r_up):
    """
    :param Q_T_H_rad: 1時間当たりの放熱器の処理暖房負荷 (MJ/h)
    :param r_up: 温水床暖房の定免法熱率 (-)
    :return: 放熱器の熱損失 (MJ/h)
    """
    return Q_T_H_rad * (1 - r_up) / r_up

### L.3 温水供給運転率

$$\large
r_{WS,rad} = \frac{Q_{T,H,rad}}{Q_{max,H,rad}}
$$

$r_{WS,rad}$：放熱器の温水供給運転率  
$Q_{T,H,rad}$：1 時間当たりの放熱器の処理暖房負荷(MJ/h)  
$Q_{max,H,rad}$：1 時間当たりの放熱器の最大暖房出力(MJ/h)  

In [ ]:
# 温水供給運転率 (2)
def get_r_WS_rad(Q_T_H_rad, Q_max_H_rad):
    """
    :param Q_T_H_rad: 1時間当たりの放熱器の処理暖房負荷 (MJ/h)
    :param Q_max_H_rad: 1時間当たりの放熱器の最大暖房出力 (MJ/h)
    :return: 温水供給運転率 (-)
    """
    return Q_T_H_rad / Q_max_H_rad

### L.4 最大暖房出力

$$\large
Q_{max,H,rad} = q_{max,fh} \times A_{f} \times \frac{\theta_{SW} - 20}{60 - 20} \times 3600 \times 10^{-6}
$$

$Q_{max,H,rad}$：1 時間当たりの放熱器の最大暖房出力(MJ/h)  
$q_{max,fh}$：温水床暖房の単位面積当たりの上面最大放熱能力(W/m<sup>2</sup>)  
$A_{f}$：温水床暖房の敷設面積(m<sup>2</sup>)  
$\theta_{SW}$：1 時間平均の往き送水温度(℃)  

In [ ]:
# 放熱器の最大暖房出力 (3)
def get_Q_max_H_rad(Theta_SW, A_f):
    """
    :param Theta_SW: 1時間平均の往き温水温度 (℃)
    :param A_f: 温水床暖房の敷設面積 (m2)
    :return: 放熱器の最大暖房出力 (MJ/h)
    """

    # 温水床暖房の単位面積当たりの上面最大放熱能力 (W/m2)
    q_max_fh = get_q_max_fh()

    Q_max_H_rad = np.ones(24*365) * q_max_fh * A_f * (Theta_SW - 20) / (60 - 20) * 3600 * 10 ** (-6)

    return Q_max_H_rad

温水床暖房の単位面積当たりの上面最大放熱能力$q_{max,fh}$は162W/m<sup>2</sup> とする。 

In [ ]:
# 温水床暖房の単位面積当たりの上面最大放熱能力 (W/m2)
def get_q_max_fh():
    return 162

### L.5 敷設面積

$$\large
A_{f} = A_{HCZ} \times r_{Af}
$$

$A_{f}$：温水床暖房の敷設面積(m<sup>2</sup>)  
$A_{HCZ}$：暖冷房区画の床面積(m<sup>2</sup>)  
$r_{Af}$：温水床暖房の敷設率  

In [ ]:
# 温水床暖房の敷設面積 (4)
def get_A_f(A_HCZ, r_Af):
    """
    :param A_HCZ: 温水床暖房の敷設面積 (m2)
    :param r_Af: 暖冷房区画の床面積 (m2)
    :return: 温水床暖房の敷設面積 (m2)
    """
    return A_HCZ * r_Af

In [ ]:
# 温水床暖房の敷設率 (5)
def get_r_f(A_f, A_HCZ):
    """
    :param A_f: 温水床暖房の敷設面積 (m2)
    :param A_HCZ: 温水床暖房の敷設面積 (m2)
    :return: 温水床暖房の敷設率 (-)
    """
    r_f = A_f / A_HCZ

    # 1000分の1未満の端数を切り下げた小数第三位までの値とする
    r_f = floor(r_f * 1000) / 1000

    return r_f

### L.6 上面放熱率

省略

In [ ]:
# 上面放熱率 (6)
def get_r_up(is_domayuka):
    # 土間床に設置された温水床暖房の上面放熱率は0.90とする
    if is_domayuka:
        return 0.90

    else:
        # R_si + R_U
        R_si_plus_R_U = get_R_si_plus_R_U()

        # (6)
        r_up = 1 - H * R_si_plus_R_U * R_U

        # 100 分の 1 未満の端数を切り捨てした小数点第 二位までの値
        return floor(r_up * 100) / 100

In [ ]:
# 1) 床暖房パネルの床上側表面熱伝達抵抗と床暖房パネル内の配管から床仕上げ材上側表面までの熱抵抗の合計
def get_R_si_plus_R_U():
    return 0.269

In [ ]:
# 2) 温度差係数U
def get_H(region, is_adjacent_not_insulated):
    # TODO: 第3章第2節への参照
    if is_adjacent_not_insulated:
        raise NotImplementedError()
    else:
        if region in [1, 2, 3]:
            return 0.05
        elif region in [4, 5, 6, 7]:
            return 0.15
        else:
            raise NotImplementedError()

### L.7 複数の温水床暖房が設置される場合の仕様の決定方法

省略